# Data Cleaning & Preprocessing
## ✅ EDA (Exploratory Data Analysis)
## ✅ Groupby, Pivot Tables
## ✅ Handling Dates & Time
## ✅ Feature Engineering
## ✅ SQL-style queries with pandas
## ✅ Visualizations with matplotlib/seaborn
## ✅ Report creation using Tableau
## ✅ Storytelling with Data
## ✅ Exporting Insights to CSV or PDF 

# 🔍 Step 1: Data Cleaning & Basic Exploration


In [1]:
import pandas as pd

In [3]:
customers = pd.read_csv("dataset/customers.csv")
transactions = pd.read_csv("dataset/transactions.csv")

In [4]:
print(customers.head())
print(transactions.head())

  customer_id             name  age gender  income   join_date  credit_limit  \
0       C1000  Michael Charles   56  Other   35795  16-12-2020          5000   
1       C1001     Ronald Boyle   46  Other  123694  18-06-2021         20000   
2       C1002   Daniel Sanders   60   Male  139879  12-06-2022         10000   
3       C1003      Billy Baker   38   Male   74886  23-11-2020         50000   
4       C1004    Ryan Williams   36  Other  148214  17-10-2020         30000   

    status  
0   Active  
1   Active  
2  Churned  
3   Active  
4   Active  
                         transaction_id customer_id transaction_date  amount  \
0  32d31488-bb51-41c6-9178-f773617d996f       C1080       22-03-2025    6.35   
1  29143434-a9d1-4672-9ebb-3ac6cd524093       C1578       10-05-2023  214.48   
2  ea841a3c-498a-42a8-9093-d3a577ddf7ad       C1700       14-11-2024   72.58   
3  bc7a9d7c-6e3e-49cb-8bb9-a914bfe9add7       C1415       29-08-2023  236.99   
4  c97d10ea-ad42-4307-8a38-63705a0fe7ed  

In [ ]:
print(customers.isnull().sum())

In [ ]:
print(transactions.isnull().sum())

In [ ]:
print(customers.dtypes)
print(transactions.dtypes)

In [ ]:
customers['join_date'] = pd.to_datetime(customers['join_date'], dayfirst=True)

In [ ]:
print(customers.dtypes)

In [ ]:
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'], dayfirst = True)

# 📊 Step 2: Exploratory Data Analysis (EDA)


In [ ]:
categorySpend = transactions.groupby('category')['amount'].sum().sort_values(ascending = True)

In [ ]:
categorySpend

In [ ]:
transactions['month'] = transactions['transaction_date'].dt.to_period('M')

In [ ]:
monthly_spend = transactions.groupby('month')['amount'].sum().sort_values(ascending = True)

In [ ]:
monthly_spend_Line = monthly_spend.plot(kind = 'line' , title = 'Monthly Spends')
monthly_spend_Line

In [ ]:
top_customers = transactions.groupby('customer_id')['amount'].sum().sort_values(ascending = False).head(7)
top_customers

In [ ]:
top_customer_name = top_customers.reset_index().merge(customers[['customer_id', 'name']], on = 'customer_id')
top_customer_name

# 🧠 Step 3: Feature Engineering


In [ ]:
agg_data = transactions.groupby('customer_id').agg(
    total_spend=('amount','sum'),
    avg_spend= ('amount','mean'),
    txn_count=('amount','count'),
    int_txn_pct=('is_international', lambda x: (x== 'Yes').mean() * 100)
).reset_index()

agg_data

In [ ]:
customer_summary = pd.merge(customers, agg_data, on='customer_id')
customer_summary

# 🔥 Step 4: Business Logic-Based Flags

In [ ]:
def customerValue(t):
    if t >= 1000:
        return 'High'
    elif t > 500 and t < 1000:
        return 'Medium'
    else: 
        return 'Low'

customer_summary['value_segment'] = customer_summary['total_spend'].apply(customerValue)
customer_summary

# Category-wise Spends

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
sortedSpend = categorySpend.sort_values(ascending = False)
colors = sns.color_palette('Set2', n_colors=len(sortedSpend))


plt.figure(figsize=(10,8))
sns.barplot(x= sortedSpend.index, y= sortedSpend.values, hue=sortedSpend.index, palette=colors)
plt.xticks(rotation=45)
plt.grid(True)
plt.title('Total Spend By Category')
plt.xlabel('Category')
plt.ylabel('Spend')
plt.tight_layout()
plt.show()

# Customer Segmentation

In [ ]:
customer_summary['value_segment'].value_counts().plot.pie(autopct="%1.1f%%", startangle=90, title="Customer Segment", figsize=(6,6))

plt.show()

# International TXN Ratio

In [ ]:

intRatio = transactions["is_international"].value_counts(normalize=True) * 100
colors = sns.color_palette('Set2', n_colors=len(intRatio))
intRatio.plot(kind='bar',title='Domestic vs International Txn', color=colors)
plt.grid(True)

# Customer Churn Prediction

In [ ]:
churnThreshold = transactions['transaction_date'].max() - pd.DateOffset(months = 3)
churnThreshold

In [ ]:
last_txn_per_customer = transactions.groupby('customer_id')['transaction_date'].max().reset_index()
last_txn_per_customer['churned'] = last_txn_per_customer['transaction_date'] > churnThreshold

In [ ]:
last_txn_per_customer

In [ ]:
customer_summary = customer_summary.merge(last_txn_per_customer[['customer_id','churned']], on='customer_id')
customer_summary

In [ ]:
from datetime import datetime

customer_summary['days_since_joining'] = (last_date - customer_summary['join_date']).dt.days

In [ ]:
features = customer_summary[['total_spend', 'avg_spend', 'txn_count', 'int_txn_pct','days_since_joining']]
target = customer_summary['churned']


target , features

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(n_estimators = 100, random_state=42)
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)

print(confusion_matrix(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt

feat_importance = pd.Series(model.feature_importances_, index=features.columns)
colors = sns.color_palette("Set2", n_colors= len(features.value_counts()))
feat_importance.sort_values().plot(kind='barh', title='Feature Importance', color = colors)
plt.tight_layout()
plt.show()


In [ ]:
sns.boxplot(x=target, y = features['txn_count'])
plt.title('avg_spend V/s Churn')
plt.xlabel('Churned')
plt.ylabel('avg_spend')
plt.grid()
plt.show()